In [ ]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import numpy as np
from tqdm import tqdm

def load_model(model_name, df):
    df['corpus_vector'] = ''
    # corpus_vector = pd.DataFrame([columns=])
    model= Doc2Vec.load(model_name)
    for i in range(len(df)):
        df['corpus_vector'].iloc[i] = model.infer_vector(df['Content'].iloc[i].split())
    return df

def dbscan(df):
    """Function to form dbscan clusters and display them"""
#     eps = 0.005# how close points should be to each other to be considered a part of a cluster 
#     min_samples = 3# the minimum number of points to form a dense region  
#     dbscan = DBSCAN( eps=eps, min_samples=min_samples,metric = "cosine" ) 
#     dbscan_model = dbscan.fit(corpus_vector)
    
    pca = PCA(n_components=2)
    result = pca.fit_transform(df['corpus_vector'].to_list())
    print(result.shape)
    db = DBSCAN(eps=0.002, min_samples=3)
    dbscan_model = db.fit(result)
    #Forming the clusters

    core_samples_mask = np.zeros_like(dbscan_model.labels_, dtype=bool)
    core_samples_mask[dbscan_model.core_sample_indices_] = True
    labels1 = dbscan_model.labels_
    print(labels1)
    n_clusters_ = len(set(labels1)) - (1 if -1 in labels1 else 0) # Number of clusters in labels
    # print(labels1)
    print(len(labels1))
    # print(n_clusters_) # number of clusters
    df['ClusterNum'] =''
    clusters1 = {} # a dictionary for different cluster 
    for c, i in enumerate(labels1):
        if i == -1:
            continue
        elif i in clusters1:
            clusters1[i].append(df['ID'].iloc[c])
            df['ClusterNum'].iloc[c] = i
        else:
            clusters1[i] = [df['ID'].iloc[c]]
            
            df['ClusterNum'].iloc[c] = i
    with open('./cluster.txt', 'w') as f:
        for c in clusters1:
            f.write("Cluster No."+" "+str(c)+" "+str(clusters1[c]))
            f.write('--------------\n')
            f.write('\n')  
    # for c in clusters1: # print the different clusters
    #     print("Cluster No."+" "+str(c)+" "+str(clusters1[c]))
    #     print()
    
    return df, clusters1

In [ ]:

df = pd.read_csv('/Users/stillssi/Desktop/NEWS_TEAM_4/MDL/content_based/sample_df.csv')
df = load_model("/Users/stillssi/Desktop/NEWS_TEAM_4/MDL/content_based/model/trainedMDL",df)


In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df, cluster1 = dbscan(df)

In [ ]:
df.drop(['CosSimilarity'], axis=1, inplace=True)
df_1 = df[df['ClusterNum'] != '']
cluster_df = pd.DataFrame(data = [cluster1])
cluster_df=cluster_df.transpose()

In [ ]:
cluster_df.to_csv('./cluster_list.csv')

In [ ]:
cluster_df = pd.read_csv('/Users/stillssi/Desktop/NEWS_TEAM_4/MDL/content_based/cluster_list.csv')

In [ ]:
cluster_df.columns

In [ ]:
cluster_df.rename(columns={'Unnamed: 0':'cluster'}, inplace=True)
cluster_df.rename(columns={'0':'list'}, inplace=True)

In [ ]:
cluster_df.set_index('cluster')

In [ ]:
df = df[df['ClusterNum'] != '']

In [ ]:
#df에서 ClusterNum 칼럼 값 가져옴 -> cluster_df칼럼 (df벡터와 cluster_df의 리스트의 뉴스ID의 벡터와 cos계산)

In [ ]:
len(cluster1)

In [ ]:
df.head(3)

In [ ]:
df = df.set_index('ID')

In [ ]:
df

In [ ]:
df['cossim_list'] = ''

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
from tqdm import tqdm
import itertools
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(df)):
    cosine_sim_list = []
    # 뉴스 아이디 리스트
    id_list = cluster_df.loc[cluster_df['cluster'] == df['ClusterNum'].iloc[i], 'list']
    tmp = id_list.values[0].replace('[','')
    tmp = tmp.replace(']','')
    tmp = tmp.split(',')
    
    id_list = list(map(int, tmp))

    id_list_df = df.loc[id_list]
    # print(id_list_df)
    vec1 = df['corpus_vector'].iloc[i]
    vec1 = np.reshape(vec1,(100,1))

    id_list_df = id_list_df['corpus_vector'].apply(lambda x: np.reshape(x, (100,1)))
    for j in id_list_df.index:
        cosine_result = cos_sim(vec1.T, id_list_df[j])
        cosine_result = cosine_result.tolist()
        if cosine_result[0][0] == 1.0:
            pass
        else:
            cosine_sim_list.append((j,cosine_result[0][0]))
    
    df['cossim_list'].iloc[i] = cosine_sim_list


In [ ]:
for i in df['cossim_list']:
    i.sort(key=lambda x: (x[0], -x[1]), reverse=True)


In [ ]:
df.loc[889553]

In [168]:
df.to_csv('./content_based_df.csv')